In [1]:
import numpy as np
import pandas as pd
import torch

torch.set_float32_matmul_precision('high')

DEV_MODE=False
PATH_COLLECTION_DATA = 'data/subtask_4b/subtask4b_collection_data.pkl'

df_collection = pd.read_pickle(PATH_COLLECTION_DATA)
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [2]:
query_variant = 'dev' if DEV_MODE else 'test'

PATH_QUERY_DATA = f'data/subtask_4b/subtask4b_query_tweets_{query_variant}.tsv'
df_query = pd.read_csv(PATH_QUERY_DATA, sep = '\t')
df_query.head()

,post_id,tweet_text
0,1,A recent research study published yesterday cl...
1,2,"""We should track the long-term effects of thes..."
2,3,"the agony of ""long haul"" covid-19 symptoms."
3,4,Home and online monitoring and assessment of b...
4,5,"it may be a long one, folks! to avoid exceedin..."


In [3]:
PARTIAL_PREDICTION_FILE = "partial-predictions/sbert/predictions.tsv"
partial_predictions = pd.read_csv(PARTIAL_PREDICTION_FILE, sep = '\t')
partial_predictions.head()

,post_id,preds
0,1,"['qgwu9fsk', '8zufbeuz', 'b5yrfllu', 'rbgoabfk..."
1,2,"['e4xw6gyp', '9f0bjknz', 'k7vjqzx1', 'hunepli1..."
2,3,"['m3m2n3fw', 'dikrttn6', 'trrg1mnw', '5351oj7i..."
3,4,"['ru2ty1y9', 'enlj85zc', 'r4q0zqam', 'xs0kxf25..."
4,5,"['n0uy6hd2', 'x9veckdi', 'qnuphz59', 'n9zqc1gm..."


In [4]:
from tqdm import tqdm
from rankers.cross_embedding_ranker import CrossRanker
from os import listdir

tqdm.pandas()

dir_list = listdir("models/cross-embedding")
dir_list.sort()

latest_checkpoint = dir_list[-1]

cross_model_name = f"models/cross-embedding/{latest_checkpoint}"
cross_ranker = CrossRanker(cross_model_name)


def get_top_cord_uids(query):
    tweet_id = query["post_id"]
    selected_docs_uids = eval(partial_predictions[partial_predictions["post_id"] == tweet_id]["preds"].values[0])[:100]

    reduced_corpus = df_collection[df_collection['cord_uid'].isin(selected_docs_uids)]
    cord_uids = reduced_corpus[:]['cord_uid'].tolist()

    doc_scores = cross_ranker.get_scores(query["tweet_text"], reduced_corpus)
    indices = np.argsort(-doc_scores)[:200]
    cross_topk = [cord_uids[x] for x in indices]

    return cross_topk

# Retrieve topk candidates using the BM25 model
df_query['cross'] = df_query.progress_apply(lambda x: get_top_cord_uids(x), axis=1)

100%|██████████| 1446/1446 [11:21<00:00,  2.12it/s]


In [5]:
from eval_scripts.eval import get_performance_mrr, get_avg_gold_in_pred, create_pred_file
if DEV_MODE:
    mrr_results = get_performance_mrr(df_query, 'cord_uid', 'cross')
    gold_results = get_avg_gold_in_pred(df_query, 'cord_uid', 'cross', list_k=[10])
    # Printed MRR@k results in the following format: {k: MRR@k}
    print(">>>")
    print(mrr_results)
    print(gold_results)
    print("<<<")

create_pred_file(df_query, "cross", prediction_size=10, include_gold=DEV_MODE, base_folder="partial-predictions/classifier")